In [1]:

#Data Preprocessing
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
# Load the data
data = pd.read_csv('continuous dataset.csv')

data

,datetime,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav,Holiday_ID,holiday,school
0,03-01-2015 01:00,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0
1,03-01-2015 02:00,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,0.016509,0.087646,5.572471,0,0,0
2,03-01-2015 03:00,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,0.016479,0.078735,5.871184,0,0,0
3,03-01-2015 04:00,889.9538,25.957544,0.018890,0.016273,22.110481,23.238794,0.017128,0.002599,10.518620,22.512231,0.016487,0.068390,5.883621,0,0,0
4,03-01-2015 05:00,893.6865,25.973840,0.018981,0.017281,21.186089,23.075403,0.017059,0.001729,9.733589,22.481653,0.016456,0.064362,5.611724,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48043,26-06-2020 20:00,1128.5592,27.246545,0.020303,0.055511,9.289304,25.715295,0.019746,0.121552,1.990773,23.746545,0.018381,0.150879,2.444658,0,0,1
48044,26-06-2020 21:00,1112.7488,27.099573,0.020395,0.053848,9.837504,25.552698,0.019632,0.153870,2.094459,23.693323,0.018320,0.156311,2.515814,0,0,1
48045,26-06-2020 22:00,1081.5680,26.971155,0.020448,0.057251,10.262464,25.393030,0.019518,0.144531,2.396369,23.658655,0.018327,0.153259,2.800717,0,0,1
48046,26-06-2020 23:00,1041.6240,26.867487,0.020464,0.064178,10.326567,25.258112,0.019403,0.108063,2.720871,23.601862,0.018358,0.152771,3.138132,0,0,1


In [2]:
# Convert timestamp to datetime format
data['datetime'] = data['datetime'][:43776].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))

# Set the datetime as index
data.set_index('datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['nat_demand'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['nat_demand'].rolling(window=24).mean()
data['rolling_std'] = data['nat_demand'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)
train_size = int(len(data) * 0.8)
train_data = data[:train_size]/1000
test_data = data[train_size:]/1000

In [6]:
#ARIMA
from statsmodels.tsa.arima.model import ARIMA

# Fit the ARIMA model
model = ARIMA(train_data['nat_demand'], order=(5, 1, 0))
model_fit = model.fit()

# Make predictions on the test set
predictions = model_fit.predict(start=len(train_data), end=len(data)-1, typ='levels')

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(test_data['nat_demand'], predictions)
mse = mean_squared_error(test_data['nat_demand'], predictions)
rmse = np.sqrt(mse)

print('MAE:', mae)
print('MSE:', mse)
print('RMSE:', rmse)
ypred=predictions
y_test=test_data['nat_demand']
y_arima=
#For MAE
mae=np.mean(np.abs(ypred-y_test))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
def mean_absolute_percentage_error(y_test, ypred): 
    return np.mean(np.abs((y_test - ypred)/y_test))*100.
mape = mean_absolute_percentage_error(y_test, ypred)
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))

C:\Users\gulsh\anaconda32\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
C:\Users\gulsh\anaconda32\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
C:\Users\gulsh\anaconda32\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


MAE: 0.22788039756857034
MSE: 0.0813881454123888
RMSE: 0.2852860764432586
MAE = 0.23 (%)
MAPE = 16.97 (%)
RSME = 0.29 (%)


In [7]:
#SARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Fit the SARIMA model
model = SARIMAX(train_data['nat_demand'], order=(5, 1, 0), seasonal_order=(0, 1, 1, 24))
model_fit = model.fit()

# Make predictions on the test set
predictions = model_fit.predict(start=len(train_data), end=len(data)-1)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(test_data['nat_demand'], predictions)
mse = mean_squared_error(test_data['nat_demand'], predictions)
rmse = np.sqrt(mse)

print('MAE:', mae)
print('MSE:', mse)
print('RMSE:', rmse)
ypred=predictions
y_test=test_data['nat_demand']

#For MAE
mae=np.mean(np.abs(ypred-y_test))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
def mean_absolute_percentage_error(y_test, ypred): 
    return np.mean(np.abs((y_test - ypred)/y_test))*100.
mape = mean_absolute_percentage_error(y_test, ypred)
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))


C:\Users\gulsh\anaconda32\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
C:\Users\gulsh\anaconda32\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


MAE: 3.672382579216278
MSE: 17.18164031825407
RMSE: 4.145074223491549
MAE = 3.67 (%)
MAPE = 300.48 (%)
RSME = 4.15 (%)


In [8]:
#LSTM
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.losses import mean_squared_error

# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Reshape the data for LSTM
X_train = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Define the model
model = Sequential()
model.add(LSTM(64, input_shape=(1, X_train.shape[2]), return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred=model.predict(X_test)
ypred = ypred.flatten() 

#For MAE
mae=np.mean(np.abs(ypred-y_test))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
def mean_absolute_percentage_error(y_test, ypred): 
    return np.mean(np.abs((y_test - ypred)/y_test))*100.
mape = mean_absolute_percentage_error(y_test, ypred)
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))


Epoch 1/50
547/547 [==============================] - 10s 7ms/step - loss: 0.0405 - val_loss: 0.0047
Epoch 2/50
547/547 [==============================] - 3s 5ms/step - loss: 0.0030 - val_loss: 0.0026
Epoch 3/50
547/547 [==============================] - 3s 5ms/step - loss: 0.0021 - val_loss: 0.0020
Epoch 4/50
547/547 [==============================] - 3s 5ms/step - loss: 0.0018 - val_loss: 0.0023
Epoch 5/50
547/547 [==============================] - 3s 5ms/step - loss: 0.0017 - val_loss: 0.0018
Epoch 6/50
547/547 [==============================] - 2s 4ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 7/50
547/547 [==============================] - 3s 5ms/step - loss: 0.0014 - val_loss: 0.0016
Epoch 8/50
547/547 [==============================] - 3s 5ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 9/50
547/547 [==============================] - 3s 5ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 10/50
547/547 [==============================] - 3s 5ms/step - loss: 0.0012 - val_loss: 0.001

In [9]:
#CNN-LSTM
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten

# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(1))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred=model.predict(X_test)

#For MAE
mae = np.mean(np.abs(ypred.ravel() - y_test.ravel()))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test.ravel() - ypred.ravel()) / y_test.ravel())) * 100
print('MAPE = {:.2f} (%)'.format(mape))


# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))

Epoch 1/50
547/547 [==============================] - 12s 18ms/step - loss: 0.0255 - val_loss: 0.0028
Epoch 2/50
547/547 [==============================] - 11s 19ms/step - loss: 0.0022 - val_loss: 0.0019
Epoch 3/50
547/547 [==============================] - 12s 21ms/step - loss: 0.0018 - val_loss: 0.0016
Epoch 4/50
547/547 [==============================] - 10s 18ms/step - loss: 0.0016 - val_loss: 0.0034
Epoch 5/50
547/547 [==============================] - 9s 17ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 6/50
547/547 [==============================] - 10s 18ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 7/50
547/547 [==============================] - 9s 17ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 8/50
547/547 [==============================] - 10s 18ms/step - loss: 0.0013 - val_loss: 0.0016
Epoch 9/50
547/547 [==============================] - 9s 17ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 10/50
547/547 [==============================] - 10s 18ms/step - loss: 0.0012 -

In [10]:
#Attention
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Reshape the data 
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the input shape
input_shape = (X_train.shape[1], 1)

# Define the model
inputs = Input(shape=input_shape)
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
pool1 = MaxPooling1D(pool_size=2)(conv1)
lstm1 = LSTM(64, return_sequences=True)(pool1)
attention = Attention()([lstm1, lstm1])
flatten = Flatten()(attention)
dropout = Dropout(0.2)(flatten)
outputs = Dense(1)(dropout)
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='mse', optimizer=Adam())

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred = model.predict(X_test)

#For MAE
mae = np.mean(np.abs(ypred.ravel() - y_test.ravel()))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test.ravel() - ypred.ravel()) / y_test.ravel())) * 100
print('MAPE = {:.2f} (%)'.format(mape))


# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))

Epoch 1/50
547/547 [==============================] - 15s 23ms/step - loss: 0.0339 - val_loss: 0.0042
Epoch 2/50
547/547 [==============================] - 14s 25ms/step - loss: 0.0049 - val_loss: 0.0027
Epoch 3/50
547/547 [==============================] - 14s 26ms/step - loss: 0.0036 - val_loss: 0.0023
Epoch 4/50
547/547 [==============================] - 14s 25ms/step - loss: 0.0032 - val_loss: 0.0022
Epoch 5/50
547/547 [==============================] - 14s 26ms/step - loss: 0.0028 - val_loss: 0.0022
Epoch 6/50
547/547 [==============================] - 14s 26ms/step - loss: 0.0027 - val_loss: 0.0020
Epoch 7/50
547/547 [==============================] - 14s 26ms/step - loss: 0.0024 - val_loss: 0.0018
Epoch 8/50
547/547 [==============================] - 14s 26ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 9/50
547/547 [==============================] - 14s 26ms/step - loss: 0.0020 - val_loss: 0.0016
Epoch 10/50
547/547 [==============================] - 14s 26ms/step - loss: 0.001

In [11]:
#Transformer
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Define the input shape
seq_len = X_train.shape[1]
n_features = 1

# Reshape the data for Transformer
X_train = X_train.values.reshape(X_train.shape[0], seq_len, n_features)
X_test = X_test.values.reshape(X_test.shape[0], seq_len, n_features)

# Define the Transformer model
inputs = Input(shape=(seq_len, n_features))
x = inputs

# Add self-attention layers
for _ in range(2):
    x = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=32)(x, x)
    x = LayerNormalization(epsilon=1e-6)(x)
    x = Dropout(0.1)(x)
    x = tf.keras.layers.Dense(32, activation="relu")(x)

# Flatten and add output layer
x = tf.keras.layers.Flatten()(x)
outputs = Dense(1)(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the model
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred = model.predict(X_test)
#For MAE
mae = np.mean(np.abs(ypred.ravel() - y_test.ravel()))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test.ravel() - ypred.ravel()) / y_test.ravel())) * 100
print('MAPE = {:.2f} (%)'.format(mape))


# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))

Epoch 1/50
547/547 [==============================] - 68s 118ms/step - loss: 0.8959 - val_loss: 0.6115
Epoch 2/50
547/547 [==============================] - 60s 109ms/step - loss: 0.2959 - val_loss: 0.2049
Epoch 3/50
547/547 [==============================] - 57s 105ms/step - loss: 0.0894 - val_loss: 0.0768
Epoch 4/50
547/547 [==============================] - 57s 105ms/step - loss: 0.0429 - val_loss: 0.0475
Epoch 5/50
547/547 [==============================] - 58s 105ms/step - loss: 0.0371 - val_loss: 0.0421
Epoch 6/50
547/547 [==============================] - 58s 105ms/step - loss: 0.0368 - val_loss: 0.0413
Epoch 7/50
547/547 [==============================] - 58s 106ms/step - loss: 0.0368 - val_loss: 0.0411
Epoch 8/50
547/547 [==============================] - 57s 105ms/step - loss: 0.0368 - val_loss: 0.0414
Epoch 9/50
547/547 [==============================] - 57s 104ms/step - loss: 0.0368 - val_loss: 0.0413
Epoch 10/50
547/547 [==============================] - 57s 103ms/step - l

In [26]:
#CNN LeNet
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers

from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import mean_squared_error
# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']


# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LeNet model
model = models.Sequential()
model.add(layers.Conv1D(6, kernel_size=5, activation='relu',  input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(16, kernel_size=5, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(120, activation='relu'))
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(1, activation='linear'))  # Output layer with linear activation for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

# Make predictions on new data
predictions = model.predict(X_test)
ypred=predictions
print('Predictions:', predictions)
ypred = ypred.flatten()


#For MAE
mae = np.mean(np.abs(ypred - y_test))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test - ypred) / y_test)) * 100
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
mse = mean_squared_error(y_test, ypred) 
rmse = np.sqrt(mse) 
print('RMSE = {:.2f} (%)'.format(rmse))

Epoch 1/50
1094/1094 [==============================] - 4s 3ms/step - loss: 0.0130
Epoch 2/50
1094/1094 [==============================] - 3s 3ms/step - loss: 0.0011
Epoch 3/50
1094/1094 [==============================] - 3s 3ms/step - loss: 9.1178e-04
Epoch 4/50
1094/1094 [==============================] - 3s 3ms/step - loss: 8.0493e-04
Epoch 5/50
1094/1094 [==============================] - 3s 3ms/step - loss: 7.4452e-04
Epoch 6/50
1094/1094 [==============================] - 3s 3ms/step - loss: 7.1518e-04
Epoch 7/50
1094/1094 [==============================] - 3s 3ms/step - loss: 7.2373e-04
Epoch 8/50
1094/1094 [==============================] - 3s 3ms/step - loss: 6.7233e-04
Epoch 9/50
1094/1094 [==============================] - 3s 3ms/step - loss: 6.6712e-04
Epoch 10/50
1094/1094 [==============================] - 3s 3ms/step - loss: 6.3691e-04
Epoch 11/50
1094/1094 [==============================] - 3s 3ms/step - loss: 6.2104e-04
Epoch 12/50
1094/1094 [==========================

In [13]:
#CNN LeNet
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers

from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import mean_squared_error
# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']


# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LeNet model
model = models.Sequential()
model.add(layers.Conv1D(6, kernel_size=5, activation='relu',  input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(16, kernel_size=5, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(120, activation='relu'))
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(1, activation='linear'))  # Output layer with linear activation for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

# Make predictions on new data
predictions = model.predict(X_test)
ypred=predictions
print('Predictions:', predictions)
ypred = ypred.flatten()


#For MAE
mae = np.mean(np.abs(ypred - y_test))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test - ypred) / y_test)) * 100
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
mse = mean_squared_error(y_test, ypred) 
rmse = np.sqrt(mse) 
print('RMSE = {:.2f} (%)'.format(rmse))

Epoch 1/50
1094/1094 [==============================] - 3s 2ms/step - loss: 0.0062
Epoch 2/50
1094/1094 [==============================] - 2s 2ms/step - loss: 0.0012
Epoch 3/50
1094/1094 [==============================] - 3s 2ms/step - loss: 0.0010
Epoch 4/50
1094/1094 [==============================] - 3s 2ms/step - loss: 9.1120e-04
Epoch 5/50
1094/1094 [==============================] - 3s 2ms/step - loss: 8.4622e-04
Epoch 6/50
1094/1094 [==============================] - 3s 2ms/step - loss: 7.9184e-04
Epoch 7/50
1094/1094 [==============================] - 3s 2ms/step - loss: 7.2472e-04
Epoch 8/50
1094/1094 [==============================] - 3s 2ms/step - loss: 7.2905e-04
Epoch 9/50
1094/1094 [==============================] - 3s 2ms/step - loss: 7.0310e-04
Epoch 10/50
1094/1094 [==============================] - 3s 2ms/step - loss: 6.6544e-04
Epoch 11/50
1094/1094 [==============================] - 3s 2ms/step - loss: 6.4472e-04
Epoch 12/50
1094/1094 [==============================

In [24]:
#LeNet-LSTM
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers

from sklearn.metrics import mean_squared_error

# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LeNet model
model = models.Sequential()
model.add(layers.Conv1D(6, kernel_size=5, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(16, kernel_size=5, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(120, activation='relu'))
model.add(layers.Dense(84, activation='relu'))

# Add LSTM layer
model.add(layers.Reshape((7, 12)))  # Reshape the output of dense layers for compatibility with LSTM
model.add(layers.LSTM(84, return_sequences=False))

model.add(layers.Dense(1, activation='linear'))  # Output layer with linear activation for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

# Make predictions on new data
predictions = model.predict(X_test)
ypred = predictions.flatten()

# For MAE
mae = np.mean(np.abs(ypred - y_test))
print('MAE = {:.2f} (%)'.format(mae))

# For MAPE
mape = np.mean(np.abs((y_test - ypred) / y_test)) * 100
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
mse = mean_squared_error(y_test, ypred) 
rmse = np.sqrt(mse) 
print('RMSE = {:.2f} (%)'.format(rmse))


Epoch 1/50
1094/1094 [==============================] - 10s 7ms/step - loss: 0.0175
Epoch 2/50
1094/1094 [==============================] - 8s 7ms/step - loss: 0.0013
Epoch 3/50
1094/1094 [==============================] - 8s 7ms/step - loss: 0.0011
Epoch 4/50
1094/1094 [==============================] - 8s 7ms/step - loss: 0.0010
Epoch 5/50
1094/1094 [==============================] - 8s 7ms/step - loss: 8.7078e-04
Epoch 6/50
1094/1094 [==============================] - 8s 7ms/step - loss: 8.0318e-04
Epoch 7/50
1094/1094 [==============================] - 8s 7ms/step - loss: 7.8339e-04
Epoch 8/50
1094/1094 [==============================] - 8s 7ms/step - loss: 7.5102e-04
Epoch 9/50
1094/1094 [==============================] - 8s 7ms/step - loss: 7.0346e-04
Epoch 10/50
1094/1094 [==============================] - 8s 7ms/step - loss: 7.0935e-04
Epoch 11/50
1094/1094 [==============================] - 8s 7ms/step - loss: 6.7021e-04
Epoch 12/50
1094/1094 [==============================] -

In [22]:
#GoogleNet Lstm
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers

from sklearn.metrics import mean_squared_error

# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the GoogleNet-LSTM model
def inception_module(x, filters):
    branch1x1 = layers.Conv1D(filters[0], 1, activation='relu', padding='same')(x)
    
    branch3x3 = layers.Conv1D(filters[1], 1, activation='relu', padding='same')(x)
    branch3x3 = layers.Conv1D(filters[2], 3, activation='relu', padding='same')(branch3x3)
    
    branch5x5 = layers.Conv1D(filters[3], 1, activation='relu', padding='same')(x)
    branch5x5 = layers.Conv1D(filters[4], 5, activation='relu', padding='same')(branch5x5)
    
    branch_pool = layers.MaxPooling1D(3, strides=1, padding='same')(x)
    branch_pool = layers.Conv1D(filters[5], 1, activation='relu', padding='same')(branch_pool)
    
    output = layers.concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

input_shape = (X_train.shape[1], X_train.shape[2])  # Adjust input shape as per your requirements

input_layer = layers.Input(shape=input_shape)
x = layers.Conv1D(6, kernel_size=5, activation='relu', padding='same')(input_layer)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(16, kernel_size=5, activation='relu', padding='same')(x)
x = layers.MaxPooling1D(pool_size=2)(x)

# Replace fully connected layers with Inception modules
x = inception_module(x, [6, 6, 8, 2, 4, 4])
x = inception_module(x, [16, 8, 12, 4, 8, 8])

x = layers.LSTM(84, return_sequences=True)(x)
x = layers.Flatten()(x)
x = layers.Dense(1, activation='linear')(x)

model = models.Model(inputs=input_layer, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

# Make predictions
predictions = model.predict(X_test)

ypred = predictions.flatten()

# For MAE
mae = np.mean(np.abs(ypred - y_test))
print('MAE = {:.2f} (%)'.format(mae))

# For MAPE
mape = np.mean(np.abs((y_test - ypred) / y_test)) * 100
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
mse = mean_squared_error(y_test, ypred) 
rmse = np.sqrt(mse) 
print('RMSE = {:.2f} (%)'.format(rmse))


Epoch 1/100
1094/1094 [==============================] - 15s 11ms/step - loss: 0.0143
Epoch 2/100
1094/1094 [==============================] - 12s 11ms/step - loss: 0.0013
Epoch 3/100
1094/1094 [==============================] - 12s 11ms/step - loss: 0.0010
Epoch 4/100
1094/1094 [==============================] - 12s 11ms/step - loss: 8.9521e-04
Epoch 5/100
1094/1094 [==============================] - 12s 11ms/step - loss: 8.8741e-04
Epoch 6/100
1094/1094 [==============================] - 12s 11ms/step - loss: 8.2278e-04
Epoch 7/100
1094/1094 [==============================] - 12s 11ms/step - loss: 7.2968e-04
Epoch 8/100
1094/1094 [==============================] - 12s 11ms/step - loss: 7.5500e-04
Epoch 9/100
1094/1094 [==============================] - 12s 11ms/step - loss: 7.1939e-04
Epoch 10/100
1094/1094 [==============================] - 12s 11ms/step - loss: 7.3628e-04
Epoch 11/100
1094/1094 [==============================] - 12s 11ms/step - loss: 6.6153e-04
Epoch 12/100
1094/10

In [2]:
#CNN-Attention
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import mean_squared_error
# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(1))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred=model.predict(X_test)
ypred = ypred.flatten()
#For MAE
mae = np.mean(np.abs(ypred - y_test))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test - ypred) / y_test)) * 100
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
mse = mean_squared_error(y_test, ypred) 
rmse = np.sqrt(mse) 
print('RMSE = {:.2f} (%)'.format(rmse))

Epoch 1/50
547/547 [==============================] - 11s 17ms/step - loss: 0.0210 - val_loss: 0.0029
Epoch 2/50
547/547 [==============================] - 9s 17ms/step - loss: 0.0022 - val_loss: 0.0019
Epoch 3/50
547/547 [==============================] - 9s 16ms/step - loss: 0.0018 - val_loss: 0.0018
Epoch 4/50
547/547 [==============================] - 9s 16ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 5/50
547/547 [==============================] - 9s 16ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 6/50
547/547 [==============================] - 8s 15ms/step - loss: 0.0014 - val_loss: 0.0015
Epoch 7/50
547/547 [==============================] - 9s 16ms/step - loss: 0.0013 - val_loss: 0.0017
Epoch 8/50
547/547 [==============================] - 9s 16ms/step - loss: 0.0013 - val_loss: 0.0016
Epoch 9/50
547/547 [==============================] - 9s 17ms/step - loss: 0.0012 - val_loss: 0.0015
Epoch 10/50
547/547 [==============================] - 8s 15ms/step - loss: 0.0012 - val_l